Data Project

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_json, expr
from pyspark.sql.types import StructType, StructField, StringType, FloatType, BooleanType, LongType
from pyspark.sql.avro.functions import from_avro

Create spark session

In [2]:
spark = SparkSession.builder \
    .appName("BinanceDataCleaning") \
    .config("spark.jars", "/home/jovyan/work/postgresql-42.7.5.jar") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0") \
    .getOrCreate()

In [3]:
df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:9092") \
    .option("subscribe", "binance_trades") \
    .option("startingOffsets", "latest") \
    .load()

In [10]:
# Schema dữ liệu
schema = StructType([
    StructField("trade_id", LongType(), True),
    StructField("symbol", StringType(), True),
    StructField("price", FloatType(), True),
    StructField("quantity", FloatType(), True),
    StructField("time", LongType(), True),
    StructField("is_buyer_maker", BooleanType(), True)
])

In [ ]:
# Chuyển đổi dữ liệu JSON
df_parsed = df.selectExpr("CAST(value AS STRING)").select(from_json(col("value"), schema).alias("data")).select("data.*")


In [6]:
# Định dạng cột timestamp
df_transformed = df_parsed.withColumn("time", expr("cast(time/1000 as timestamp)"))

In [9]:
# Ghi vào bảng staging_transactions (Data Warehouse - PostgreSQL)
def write_to_postgres(df, epoch_id):
    df.write \
        .format("jdbc") \
        .option("url", "jdbc:postgresql://postgres:5432/crypto_db") \
        .option("dbtable", "staging_transactions") \
        .option("user", "postgres") \
        .option("password", "password") \
        .mode("append") \
        .save()

df_transformed.writeStream \
    .foreachBatch(write_to_postgres) \
    .outputMode("append") \
    .start() \
    .awaitTermination()

StreamingQueryException: [STREAM_FAILED] Query [id = 5ee319ca-616f-4646-b0fb-75cbadb09ac7, runId = 95c40d37-dbc0-47c3-b3ed-e2b690650eb7] terminated with exception: An exception was raised by the Python Proxy. Return Message: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 617, in _call_proxy
    return_value = getattr(self.pool[obj_id], method)(*params)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/pyspark/sql/utils.py", line 120, in call
    raise e
  File "/usr/local/spark/python/pyspark/sql/utils.py", line 117, in call
    self.func(DataFrame(jdf, wrapped_session_jdf), batch_id)
  File "/tmp/ipykernel_91/2523083034.py", line 10, in write_to_postgres
    .save()
     ^^^^^^
  File "/usr/local/spark/python/pyspark/sql/readwriter.py", line 1461, in save
    self._jwrite.save()
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1322

In [8]:
pg_url = "jdbc:postgresql://postgres:5432/crypto_db"
pg_properties = {
    "user": "postgres",
    "password": "password",
    "driver": "org.postgresql.Driver"
}

In [8]:
df = spark.read.jdbc(url=pg_url, table="raw_transactions", properties=pg_properties)


In [9]:
df.show(5)


+----------+-------+--------+--------+--------------------+--------------+-----------+
|  trade_id| symbol|   price|quantity|                time|is_buyer_maker|total_value|
+----------+-------+--------+--------+--------------------+--------------+-----------+
|4532361114|BTCUSDT|97529.57|  6.0E-5|2025-02-07 05:07:...|          true|  5.8517742|
|4532361115|BTCUSDT|97529.57|  6.0E-5|2025-02-07 05:07:...|          true|  5.8517742|
|4532361116|BTCUSDT|97529.38|  6.0E-5|2025-02-07 05:07:...|          true|  5.8517628|
|4532361117|BTCUSDT|97529.38|  6.0E-5|2025-02-07 05:07:...|          true|  5.8517628|
|4532361118|BTCUSDT|97529.37|  1.2E-4|2025-02-07 05:07:...|          true| 11.7035244|
+----------+-------+--------+--------+--------------------+--------------+-----------+
only showing top 5 rows



In [11]:
# Loại bỏ NULL
df = df.na.drop()

# Loại bỏ dữ liệu trùng lặp
df = df.dropDuplicates(["trade_id"])

# Chuyển đổi kiểu dữ liệu
df = df.withColumn("price", col("price").cast("double"))
df = df.withColumn("quantity", col("quantity").cast("double"))
df = df.withColumn("time", col("time").cast("timestamp"))

# Tạo cột tổng giá trị giao dịch
df = df.withColumn("total_value", expr("price * quantity"))

In [12]:
df.show(5)


+----------+-------+--------+--------+--------------------+--------------+-----------+
|  trade_id| symbol|   price|quantity|                time|is_buyer_maker|total_value|
+----------+-------+--------+--------+--------------------+--------------+-----------+
|4532361114|BTCUSDT|97529.57|  6.0E-5|2025-02-07 05:07:...|          true|  5.8517742|
|4532361115|BTCUSDT|97529.57|  6.0E-5|2025-02-07 05:07:...|          true|  5.8517742|
|4532361116|BTCUSDT|97529.38|  6.0E-5|2025-02-07 05:07:...|          true|  5.8517628|
|4532361117|BTCUSDT|97529.38|  6.0E-5|2025-02-07 05:07:...|          true|  5.8517628|
|4532361118|BTCUSDT|97529.37|  1.2E-4|2025-02-07 05:07:...|          true| 11.7035244|
+----------+-------+--------+--------+--------------------+--------------+-----------+
only showing top 5 rows



In [13]:
df.write.jdbc(url=pg_url, table="staging_transactions", mode="overwrite", properties=pg_properties)

print("Dữ liệu đã làm sạch và lưu vào Staging Layer trong Data Warehouse!")


Dữ liệu đã làm sạch và lưu vào Staging Layer trong Data Warehouse!


In [3]:
# Đọc dữ liệu từ Kafka
df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9094") \
    .option("subscribe", "binance_trades") \
    .option("startingOffsets", "latest") \
    .load()

AnalysisException: Failed to find data source: kafka. Please deploy the application as per the deployment section of Structured Streaming + Kafka Integration Guide.